<a href="https://colab.research.google.com/github/fabnancyuhp/DEEP-LEARNING/blob/main/NOTEBOOKS/introduction-to-hugging-face-transformers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

ref : https://huggingface.co/transformers/training.html<br>
ref : https://huggingface.co/docs/datasets/loading_datasets.html<br>
ref : https://blog.tensorflow.org/2019/11/hugging-face-state-of-art-natural.html<br>
In this notebook, 
* we will see a text classification implementation on the IMDB dataset with the Transfer Learning technique. 
* we will show how to fine-tune a pretrained model from the Transformers library. 
* we will fine-tune BERT on the IMDB dataset

BERT stands for Bidirectional Encoder Representations from Transformers. In TensorFlow, models can be directly trained using Keras and the fit method.<br><br>

**Hugging Face** is an NLP-focused startup with a large open-source community, in particular around the **Transformers library.** 🤗/Transformers is a python-based library that exposes an API to use many well-known transformer architectures, such as BERT, RoBERTa, GPT-2 or DistilBERT, that obtain state-of-the-art results on a variety of NLP tasks like text classification, information extraction, question answering, and text generation. Those architectures come pre-trained with several sets of weights.<br><br>

The **Transformers library** has seen super-fast growth in PyTorch and has recently been ported to TensorFlow 2.0, offering an API that now works with Keras’ fit API, TensorFlow Extended, and TPUs<br><br>

**Hugging Face transformers** is a collection of state-of-the-art NLP (Natural Language Processing) models. They offer a wide variety of architectures to choose from (BERT, GPT-2, RoBERTa etc) as well as a hub of pre-trained models uploaded by users and organisations. <br><br>
**Hugging Face Datasets** is a wrapper library that provides some tools to load and process data in many commonly used formats (CSV, JSON etc).

# Hugging Face Datasets
**Hugging Face Datasets** is a wrapper library that provides some tools to load and process data in many commonly used formats (CSV, JSON etc).<br><br>
Before we can fine-tune a model, we need a dataset. In this tutorial, we will show you how to fine-tune BERT on the IMDB dataset: the task is to classify whether movie reviews are positive or negative. Bellow, we load the IMBD data structure from the hugging-face hub. Row_datasets is DatasetDict object.

In [ ]:
#!pip install datasets
from datasets import load_dataset

raw_datasets = load_dataset("imdb")

  0%|          | 0/3 [00:00<?, ?it/s]

Here, we show how a hugging-face DatasetDict is modelized. When we run the cell below we see that a hugging-face DatasetDict is a data structure close to a python dictionary. The raw_datasets DatasetDict has 3 sub-datasets: train, test and unsupervised. The unsupervised dataset is the union of train and test datasets.<br>
To access the movies reviews from the training dataset, we have to run raw_datasets['train']['text']. To get the labels of the training dataset we run raw_datasets['train']['label']

In [ ]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
})

In [ ]:
print(raw_datasets['train']['text'][0:1][0][0:170])
print(raw_datasets['train']['text'][1:2][0][0:170])

I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967. I also heard that at first it was s
"I Am Curious: Yellow" is a risible and pretentious steaming pile. It doesn't matter what one's political views are because this film can hardly be taken seriously on any


In [ ]:
raw_datasets['train']['label'][0:2]

[0, 0]

In [ ]:
long_text = [len(x) for x in raw_datasets['train']['text']]

# Preprocess/Tokenize a hugging-face dataset
Remind the goal of this notebook is to fin-tuned a Bert cased model. To preprocess our data, we will need a tokenizer. A tokenizer splits the data in tokens (these can be characters, words, part of words). For this task, we are using the tokenizer from the pre-trained model we selected (bert-base-cased).

In [ ]:
#!pip install transformers
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")


def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)

  0%|          | 0/50 [00:00<?, ?ba/s]

The most importants features made during the tokenize step  are 'attention_mask', 'input_ids', 'label', 'token_type_ids'. Now, we don't need the text feature anymore. As a consequence, We remove the text columns from the huggingface dataset later in this notebook.

In [ ]:
#tokenized_datasets = tokenized_datasets.remove_columns(["text"])
#tokenized_datasets

# Fine-tuning the hugging-face bert-case model with tensorflow

## Making the dataset usable by a tensorflow model
* First step: we define the content of the training set and the test set.
* Second step: Second step: we make these 2 datasets compatibles with tensorflow 
* Third step: we make the training and test dataset suitables for a tensorflow model

In [ ]:
full_train_dataset = tokenized_datasets["train"]
full_test_dataset = tokenized_datasets["test"]
full_train_dataset

Dataset({
    features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 25000
})

In [ ]:
#Second step: Second step: we make these 2 datasets compatibles with tensorflow
tf_train_dataset = full_train_dataset.with_format("tensorflow")
tf_test_dataset = full_test_dataset.with_format("tensorflow")

We take a look at tf_train_dataset['input_ids']. It is tf.Tensor of shape=(25000, 512)

In [ ]:
tf_train_dataset['input_ids']

<tf.Tensor: shape=(25000, 512), dtype=int64, numpy=
array([[  101,   146, 12765, ...,     0,     0,     0],
       [  101,   107,   146, ...,     0,     0,     0],
       [  101,  1409,  1178, ...,     0,     0,     0],
       ...,
       [  101,  1188,  1273, ...,     0,     0,     0],
       [  101,   112,  1109, ...,  7880, 13225,   102],
       [  101,  1109,  1642, ...,     0,     0,     0]])>

We take a look at tf_train_dataset['attention_mask']. It is also a tf.Tensor of shape=(25000,512).

In [ ]:
tf_train_dataset['attention_mask']

<tf.Tensor: shape=(25000, 512), dtype=int64, numpy=
array([[1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       ...,
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 1, 1, 1],
       [1, 1, 1, ..., 0, 0, 0]])>

In [ ]:
#Third step: we make the training and test dataset suitables for a tensorflow model
import tensorflow as tf
train_features = {x: tf_train_dataset[x] for x in tokenizer.model_input_names}
train_tf_dataset = tf.data.Dataset.from_tensor_slices((train_features, tf_train_dataset["label"]))
train_tf_dataset = train_tf_dataset.shuffle(len(tf_train_dataset)).batch(8)

test_features = {x: tf_test_dataset[x] for x in tokenizer.model_input_names}
test_tf_dataset = tf.data.Dataset.from_tensor_slices((test_features, tf_test_dataset["label"]))
test_tf_dataset = test_tf_dataset.batch(8)

Now, we see how train_tf_dataset looks like:

In [ ]:
list(train_tf_dataset.as_numpy_iterator())[0:2]

[({'input_ids': array([[ 101, 1188, 1108, ...,    0,    0,    0],
          [ 101, 1130, 2030, ...,    0,    0,    0],
          [ 101,  146, 1294, ...,    0,    0,    0],
          ...,
          [ 101,  146, 1309, ...,    0,    0,    0],
          [ 101, 1752, 1104, ..., 2934, 1139,  102],
          [ 101,  146, 1145, ...,    0,    0,    0]]),
   'token_type_ids': array([[0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0],
          ...,
          [0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0]]),
   'attention_mask': array([[1, 1, 1, ..., 0, 0, 0],
          [1, 1, 1, ..., 0, 0, 0],
          [1, 1, 1, ..., 0, 0, 0],
          ...,
          [1, 1, 1, ..., 0, 0, 0],
          [1, 1, 1, ..., 1, 1, 1],
          [1, 1, 1, ..., 0, 0, 0]])},
  array([1, 0, 0, 0, 0, 1, 1, 1])),
 ({'input_ids': array([[  101,   146,  3535, ...,     0,     0,     0],
          [  101,  1274,   112, ...,     0,     0,   

## We import the tensorflow pretrained bert-base-cased model from the huggingface hub.
We import the pretrained bert-base-cased model from the huggingface hub. The model we will import is paired with the tokenizer we imported earlier in this notebook. We import the bert-base-cased in the cell below. We set num_labels=2 because we have 2 distincts values in the label columns from our dataset. In the cell below, model is a tensorflow model.

In [ ]:
#!pip install transformers
import tensorflow as tf
from transformers import TFAutoModelForSequenceClassification

model = TFAutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=2)

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## We fine-tune the pretrained model
We compile the tensorflow model as we always do when we use tensorflow. Here we use the
* the adam  optimizer with learning_rate=5e-5
* SparseCategoricalCrossentropy loss fucntion
* SparseCategoricalAccuracy() metrics

To fine-tune properly a tensorflow hugging-face model with an adam optimizer, it recomded to set the leraning_rate to 5e-5.

In [ ]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=5e-5),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=tf.metrics.SparseCategoricalAccuracy(),
)

model.fit(train_tf_dataset, validation_data=test_tf_dataset, epochs=2)

Epoch 1/2
3125/3125 [==============================] - 4092s 1s/step - loss: 0.2990 - sparse_categorical_accuracy: 0.8760 - val_loss: 0.2335 - val_sparse_categorical_accuracy: 0.9049
Epoch 2/2
3125/3125 [==============================] - 4072s 1s/step - loss: 0.2019 - sparse_categorical_accuracy: 0.9231 - val_loss: 0.2356 - val_sparse_categorical_accuracy: 0.9048


After the compile step, model is a fine-tuned transformer model. Then, we can apply model on a new movie review.

In [ ]:
new_movie_review = ["I was extraordinarily impressed by this film. It's one of the best sports films \
                    I've every seen. The visuals in this film are outstanding. I love the sequences \
                    in which the camer tracks the ball as it flies through the air or into the cup. \
                    The film moves well, offering both excitement and drama. The cinematography was fantastic.\
                    <br /><br />The acting performances are great. I was surprised by young Shia LaBeouf.\
                    He does well in this role. Stephen Dillane is also good as the brooding Harry Vardon. \
                    Peter Firth, Justin Ashforth, and Elias Koteas offer able support. \
                    The film is gripping and entertaining and for the first time in my \
                    life actually made me want to watch a golf tournament."]

In order to apply the model to the new review, We have to:
* vectorize the new movie review
* put the vectorized text into a tf.data.Dataset
* to apply the .batch(1) method to make review_data suitable for the fine-tuned model.

In [ ]:
#we vectorize
token_new_review = tokenizer(new_movie_review, padding="max_length", truncation=True)

#put the vectorized text into a tf.data.Dataset
import tensorflow as tf
review_data = tf.data.Dataset.from_tensor_slices((dict(token_new_review)))

#make review_data suitable for the fine-tuned model.
review_data_batch = review_data.batch(1)

Now, we can apply the fine-tuned model to review_data_batch. Then, we apply a softmax function to the result to get a probability vector.

In [ ]:
import numpy as np
import tensorflow as tf

review_sentiment = ['negatif review','positif review']

#we apply the fine-tuned model to review_data_batch
pred = model.predict(review_data_batch).logits
#we apply a sofmax function to get a probability vector
prob_vector = tf.nn.softmax(pred, axis=1).numpy()
prob_vector

1/1 [==============================] - 3s 3s/step


array([[0.00323815, 0.9967618 ]], dtype=float32)

Then, we display the predicted class of the new review

In [ ]:
import numpy as np
review_sentiment = ['negatif review','positif review']
review_sentiment[np.argmax(prob_vector)]

'positif review'